In [90]:
import pandas as pd
import numpy as np
import torch
# from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
import skorch
import skorch.helper
from skorch import NeuralNetClassifier
from skorch.dataset import Dataset

import sys  
sys.path.insert(0, '/root/Project-CS224N-ED-Disposition/')
from scripts import utils
import importlib
importlib.reload(utils)

index_col = 'Unnamed: 0'

In [2]:
df = pd.read_csv('/root/data/zero_shot_predictions/scores.csv').set_index(index_col)
df

,sequence,Edema,Opacities,Pericaridial effusion,Pneumonia,Pleural Effusion,Pneumothorax,Aspiration,Nephrolithiasis,Intraparenchymal hemorrhage,...,Cholecystitis,Intracranial hemorrhage,Stroke,Infection,Small bowel obstruction,Abdominal Aortic Aneurysm,Aortic dissection,Liver laceration,Colitis,Pancreatitis
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,1. Interval increase in perihilar opacities a...,0.929120,0.920819,0.737004,0.524612,0.185744,0.118200,0.045569,0.018194,0.014132,...,0.000431,0.000424,0.000250,0.000204,0.000184,0.000172,0.000117,0.000098,0.000074,0.000046
1,1. No acute intracranial abnormality. 2. Mil...,0.106327,0.090323,0.111382,0.000776,0.096998,0.001084,0.086613,0.028645,0.001540,...,0.018829,0.000796,0.005907,0.001479,0.000123,0.000192,0.000498,0.000157,0.011096,0.000875
2,1. Multiple loops of dilated small bowel with...,0.070339,0.741595,0.214792,0.007977,0.182043,0.072368,0.442881,0.017711,0.030738,...,0.044908,0.003074,0.008334,0.018739,0.967760,0.006287,0.003933,0.004344,0.007286,0.001242
3,1. Interval increased loculated moderate meta...,0.007522,0.441353,0.001170,0.004349,0.367867,0.008329,0.006146,0.001017,0.000746,...,0.000539,0.000267,0.000590,0.000482,0.000492,0.000297,0.000283,0.000482,0.000569,0.000209
4,1. Redemonstrated fracture of the femoral hea...,0.011500,0.378758,0.008745,0.001767,0.032812,0.004236,0.136899,0.022915,0.012002,...,0.004594,0.003910,0.008085,0.002214,0.002143,0.007413,0.001433,0.000586,0.003734,0.001374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146144,1. Acute superior endplate compression deform...,0.412637,0.565540,0.455190,0.018393,0.451436,0.060820,0.332372,0.102796,0.499176,...,0.230802,0.094018,0.164778,0.062306,0.011410,0.113403,0.036457,0.014308,0.135798,0.009234
146145,1. Diffuse anasarca and mesenteric edema. Pat...,0.909280,0.448288,0.229874,0.259759,0.905350,0.308534,0.230740,0.140058,0.238761,...,0.055338,0.068186,0.016210,0.020335,0.006509,0.030965,0.010458,0.020484,0.012868,0.002375
146146,1. No ultrasound evidence of deep venous thro...,0.012299,0.033661,0.014146,0.005479,0.021521,0.008577,0.062111,0.035728,0.032398,...,0.017090,0.013043,0.017339,0.003024,0.000778,0.061921,0.013719,0.005801,0.001874,0.000996


In [3]:
raw_train = pd.read_csv('/root/data/ed_train.csv').set_index(index_col)
raw_val = pd.read_csv('/root/data/ed_val.csv').set_index(index_col)
raw_test = pd.read_csv('/root/data/ed_test.csv').set_index(index_col)
print(len(raw_train), len(raw_val), len(raw_test))
total_rows = sum([len(raw_train), len(raw_val), len(raw_test)])
print(total_rows)
assert total_rows == len(df)

102304 29230 14615


In [6]:
train_df = df.join(raw_train, how='right')
val_df = df.join(raw_val, how='right')
test_df = df.join(raw_test, how='right')
print(len(train_df), len(val_df), len(test_df))
assert len(train_df) == len(raw_train)
assert len(val_df) == len(raw_val)
assert len(test_df) == len(raw_test)

102304 29230 14615


In [129]:
X_train = train_df[utils.labels].values.astype(np.float32)
X_val = val_df[utils.labels].values.astype(np.float32)
X_test = test_df[utils.labels].values.astype(np.float32)

y_col = "ED_dispo"
y_train = train_df[[y_col]]
y_val = val_df[[y_col]]
y_test = test_df[[y_col]]

input_dim = X_train.shape[1]
output_dim = len(y_train[y_col].unique())
print(input_dim, output_dim)

label_enc = OrdinalEncoder()
label_enc.fit(y_train)
print(y_train[y_col].unique())
print(label_enc.categories_)

y_train = label_enc.transform(y_train).flatten().astype(np.int64)
y_val = label_enc.transform(y_val).flatten().astype(np.int64)
y_test = label_enc.transform(y_test).flatten().astype(np.int64)
y_train

31 4
['Admit to Inpatient' 'Discharge' 'Place in Observation-CDU'
 'Place in Observation']
[array(['Admit to Inpatient', 'Discharge', 'Place in Observation',
       'Place in Observation-CDU'], dtype=object)]


array([0, 1, 0, ..., 1, 0, 1])

In [169]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Network(nn.Module):
  def __init__(self, input_dim=31, output_dim=4, hidden_dim=64, dropout_prob=0.2, hidden_layers=1):
    super(Network, self).__init__()
    self.hidden_layers = hidden_layers
    self.dropout_in = nn.Dropout(dropout_prob)
    self.fc_in = nn.Linear(input_dim, hidden_dim)
    self.fc_out = nn.Linear(hidden_dim, output_dim)
    self.softmax = nn.Softmax(dim=-1)
    if hidden_layers > 1:
        self.dropout_mid = nn.Dropout(dropout_prob)
        self.fc_mid = nn.Linear(hidden_dim, hidden_dim)

  def forward(self, x):
    x = self.dropout_in(x)
    x = self.fc_in(x)
    x = F.gelu(x)
    
    if self.hidden_layers > 1:
        x = self.dropout_mid(x)
        x = self.fc_mid(x)
        x = F.gelu(x)

    x = self.fc_out(x)
    x = self.softmax(x)
    return x

n = Network(hidden_dim=32, dropout_prob=0.5, hidden_layers=2)

In [170]:
n(torch.from_numpy(X_test)).shape

torch.Size([14615, 4])

In [171]:
>>> def ds_accuracy(net, ds, y=None):
...     # assume ds yields (X, y), e.g. torchvision.datasets.MNIST
...     y_true = [y for _, y in ds]
...     y_pred = net.predict(ds)
...     return sklearn.metrics.accuracy_score(y_true, y_pred)
>>> net = MyNet(callbacks=[
...     EpochScoring(ds_accuracy, use_caching=False)])
>>> ds = torchvision.datasets.MNIST(root=mnist_path)
>>> net.fit(ds)

tensor([[0.2582, 0.2633, 0.2422, 0.2363],
        [0.2594, 0.2656, 0.2369, 0.2381],
        [0.2582, 0.2637, 0.2418, 0.2364],
        ...,
        [0.2612, 0.2627, 0.2448, 0.2313],
        [0.2584, 0.2635, 0.2415, 0.2366],
        [0.2603, 0.2607, 0.2401, 0.2389]], grad_fn=<SoftmaxBackward0>)

In [176]:
net = skorch.NeuralNet(
    Network,
    criterion=nn.CrossEntropyLoss,
    train_split=skorch.helper.predefined_split(Dataset(X_val, y_val)),
    iterator_train__shuffle=True, # Shuffle training data on each epoch
    callbacks=[skorch.callbacks.EpochScoring(lambda a, b, c: 44)]
)

net.fit(X_train, y_train)
# y_proba = net.predict_proba(train_data.X)

  epoch    <lambda>    train_loss    valid_loss     dur
-------  ----------  ------------  ------------  ------
      1          44        1.3325        1.2846  1.1581
      2          44        1.2565        1.2359  1.1578
      3          44        1.2232        1.2137  1.1582
      4          44        1.2068        1.2015  1.1553
      5          44        1.1972        1.1936  1.1571
      6          44        1.1902        1.1877  1.1590
      7          44        1.1851        1.1829  1.1567
      8          44        1.1809        1.1789  1.1577
      9          44        1.1772        1.1756  1.1573
     10          44        1.1744        1.1727  1.1546


<class 'skorch.net.NeuralNet'>[initialized](
  module_=Network(
    (dropout_in): Dropout(p=0.2, inplace=False)
    (fc_in): Linear(in_features=31, out_features=64, bias=True)
    (fc_out): Linear(in_features=64, out_features=4, bias=True)
    (softmax): Softmax(dim=-1)
  ),
)

In [177]:
from sklearn.model_selection import GridSearchCV

#   def __init__(self, input_dim=31, output_dim=4, hidden_dim=64, dropout_prob=0.2, hidden_layers=1):
# deactivate skorch-internal train-valid split and verbose logging
net = skorch.NeuralNet(
    Network,
    criterion=nn.CrossEntropyLoss,
#     train_split=skorch.helper.predefined_split(Dataset(X_val, y_val)),
    iterator_train__shuffle=True, # Shuffle training data on each epoch
    callbacks=[skorch.callbacks.EpochScoring(lambda a, b, c: 44)],
)

params = {
#     'lr': [1e-2, 1e-3],
    'max_epochs': [1, 2],
#     'max_epochs': [1, 10, 100],
#     'module__hidden_dim': [16, 64, 256],
#     'module__dropout_prob': [0, 0.2, 0.4],
#     'module__hidden_layers': [1, 2],
    
}
gs = GridSearchCV(net, params, refit=True, cv=2, scoring='accuracy', verbose=2)

gs.fit(X_train, y_train)
print("best score: {:.3f}, best params: {}".format(gs.best_score_, gs.best_params_))

Fitting 2 folds for each of 2 candidates, totalling 4 fits
  epoch    <lambda>    train_loss    valid_loss     dur
-------  ----------  ------------  ------------  ------
      1          44        1.3630        1.3395  0.4660


/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 192, in wrapper
    return func(*args, **kwargs)
  File "/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/metrics/_classification.py",

[CV] END .......................................max_epochs=1; total time=   0.9s
  epoch    <lambda>    train_loss    valid_loss     dur
-------  ----------  ------------  ------------  ------
      1          44        1.3540        1.3310  0.4662


/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 192, in wrapper
    return func(*args, **kwargs)
  File "/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/metrics/_classification.py",

[CV] END .......................................max_epochs=1; total time=   0.9s
  epoch    <lambda>    train_loss    valid_loss     dur
-------  ----------  ------------  ------------  ------
      1          44        1.3653        1.3406  0.4662
      2          44        1.3202        1.3010  0.4644


/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 192, in wrapper
    return func(*args, **kwargs)
  File "/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/metrics/_classification.py",

[CV] END .......................................max_epochs=2; total time=   1.4s
  epoch    <lambda>    train_loss    valid_loss     dur
-------  ----------  ------------  ------------  ------
      1          44        1.3455        1.3252  0.4644
      2          44        1.3070        1.2925  0.4644


/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 192, in wrapper
    return func(*args, **kwargs)
  File "/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/metrics/_classification.py",

[CV] END .......................................max_epochs=2; total time=   1.4s
  epoch    <lambda>    train_loss    valid_loss     dur
-------  ----------  ------------  ------------  ------
      1          44        1.3357        1.2940  0.9531
best score: nan, best params: {'max_epochs': 1}
